# Задание из четвёртой лабы

Обучите один из алгоритмов классификации из библиотеки sklearn. Объяснять принцип работы классификатора в этой лабораторной не требуется

1. Выберите один из датасетов первой лабораторной


2. Найдите в данных признак, который будете предсказывать
    
    Если это категория, то у неё должно быть от 2 до 10 значений (например колонка species в датасете миграции птиц содержит 3 категории)
    
    Если признак количественный, то его нужно дискретизировать (например для популярности трека в датасете spotify можно разделить треки на 3 категории: 0-20 - популярные, 21-50 - обычные, 51 - 100 - непопулярные). Полученную категорию записать в датафрейм как отдельную колонку


3. Выделите признаки для предсказания
    
    Используйте от 2 до 5 количественных признаков для предсказания целевой переменной


4. Проверьте нормальность количественных признаков
    
    Используйте статистический тест из лекции. Если распределение является лог-нормальным, логарифмируйте его и проверьте на нормальность
    
    Как минимум 1 из признаков должен быть нормальным


5. Покажите качество предсказания модели с помощью метрик

    Метрика должна учитывать баланс классов в датасете

In [1]:
import pandas as pd
import numpy as np

In [11]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Датасет: https://www.kaggle.com/isaienkov/nba2k20-player-dataset

In [13]:
df = pd.read_csv('nba2k20-full.csv')

In [14]:
df.head()

,full_name,rating,jersey,team,position,b_day,height,weight,salary,country,draft_year,draft_round,draft_peak,college
0,LeBron James,97,#23,Los Angeles Lakers,F,12/30/84,6-9 / 2.06,250 lbs. / 113.4 kg.,$37436858,USA,2003,1,1,NaN
1,Kawhi Leonard,97,#2,Los Angeles Clippers,F,06/29/91,6-7 / 2.01,225 lbs. / 102.1 kg.,$32742000,USA,2011,1,15,San Diego State
2,Giannis Antetokounmpo,96,#34,Milwaukee Bucks,F-G,12/06/94,6-11 / 2.11,242 lbs. / 109.8 kg.,$25842697,Greece,2013,1,15,NaN
3,Kevin Durant,96,#7,Brooklyn Nets,F,09/29/88,6-10 / 2.08,230 lbs. / 104.3 kg.,$37199000,USA,2007,1,2,Texas
4,James Harden,96,#13,Houston Rockets,G,08/26/89,6-5 / 1.96,220 lbs. / 99.8 kg.,$38199000,USA,2009,1,3,Arizona State


Будем предсказывать позицию игрока на поле, position

Для предсказания будем использовать признаки rating, height, weight, salary

In [15]:
cols = ['rating', 'height', 'weight', 'salary']

In [16]:
X = df[cols]

In [17]:
X

,rating,height,weight,salary
0,97,6-9 / 2.06,250 lbs. / 113.4 kg.,$37436858
1,97,6-7 / 2.01,225 lbs. / 102.1 kg.,$32742000
2,96,6-11 / 2.11,242 lbs. / 109.8 kg.,$25842697
3,96,6-10 / 2.08,230 lbs. / 104.3 kg.,$37199000
4,96,6-5 / 1.96,220 lbs. / 99.8 kg.,$38199000
...,...,...,...,...
424,68,6-0 / 1.83,180 lbs. / 81.6 kg.,$522738
425,68,6-3 / 1.91,200 lbs. / 90.7 kg.,$79568
426,68,6-10 / 2.08,200 lbs. / 90.7 kg.,$79568
427,68,6-3 / 1.91,190 lbs. / 86.2 kg.,$898310


In [18]:
y = df['position']

In [19]:
y

0        F
1        F
2      F-G
3        F
4        G
      ... 
424      G
425      G
426      F
427      G
428      F
Name: position, Length: 429, dtype: object

In [20]:
X.isna().sum()

rating    0
height    0
weight    0
salary    0
dtype: int64

In [21]:
X.loc[:, 'height'] = X.loc[:, 'height'].apply(lambda x : float(x.split('/')[1]))

/Users/maria/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [97]:
X.loc[:, 'weight'] = X.loc[:, 'weight'].apply(lambda x : float((x.split("/")[1]).split(" ")[1]))

<ipython-input-97-fca34bf418d2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['weight'] = X['weight'].apply(lambda x : float((x.split("/")[1]).split(" ")[1]))


In [98]:
X['weight'].head()

0    113.4
1    102.1
2    109.8
3    104.3
4     99.8
Name: weight, dtype: float64

In [74]:
("225 lbs. / 102.1 kg.".split("/")[1]).split(" ")[1]

'102.1'

In [101]:
X.loc[:, 'salary'] = X.loc[:, 'salary'].apply(lambda x : int(x[1:]))
#X['salary'] = X['salary'].apply(lambda x : int(x[1:]))

<ipython-input-101-7e0f75aa88e8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['salary'] = X['salary'].apply(lambda x : int(x[1:]))


In [102]:
X

,rating,height,weight,salary
0,97,2.06,113.4,37436858
1,97,2.01,102.1,32742000
2,96,2.11,109.8,25842697
3,96,2.08,104.3,37199000
4,96,1.96,99.8,38199000
...,...,...,...,...
424,68,1.83,81.6,522738
425,68,1.91,90.7,79568
426,68,2.08,90.7,79568
427,68,1.91,86.2,898310


In [103]:
from sklearn.preprocessing import LabelEncoder

In [104]:
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

In [105]:
y

array([2, 2, 4, 2, 5, 5, 3, 2, 5, 0, 5, 0, 5, 5, 3, 5, 0, 2, 5, 5, 6, 3,
       5, 5, 6, 5, 5, 3, 5, 2, 2, 5, 5, 0, 0, 2, 5, 0, 5, 2, 5, 5, 0, 2,
       1, 5, 5, 2, 5, 2, 5, 2, 5, 5, 2, 6, 5, 2, 2, 2, 3, 0, 2, 0, 0, 0,
       5, 3, 3, 5, 3, 2, 2, 2, 2, 2, 0, 3, 0, 0, 5, 2, 2, 2, 5, 0, 2, 5,
       5, 2, 5, 3, 0, 5, 2, 5, 2, 0, 5, 5, 5, 2, 5, 0, 0, 0, 5, 2, 2, 5,
       2, 2, 5, 5, 5, 5, 3, 0, 5, 5, 5, 2, 5, 6, 5, 2, 5, 2, 0, 4, 2, 3,
       5, 5, 5, 5, 2, 5, 2, 2, 0, 4, 5, 4, 2, 2, 2, 2, 5, 5, 0, 2, 5, 3,
       1, 1, 3, 2, 2, 5, 5, 1, 0, 5, 1, 4, 5, 5, 2, 2, 5, 2, 5, 5, 2, 0,
       0, 4, 2, 2, 5, 2, 2, 2, 3, 2, 2, 0, 5, 5, 0, 5, 2, 5, 6, 5, 5, 0,
       5, 5, 2, 5, 5, 0, 0, 5, 5, 0, 4, 4, 5, 5, 5, 5, 5, 2, 4, 5, 2, 2,
       2, 5, 2, 2, 2, 0, 2, 0, 5, 5, 5, 2, 5, 4, 5, 2, 2, 6, 2, 2, 3, 0,
       6, 5, 0, 5, 5, 5, 5, 3, 5, 5, 3, 2, 4, 5, 2, 5, 3, 3, 2, 5, 0, 3,
       2, 5, 3, 2, 5, 3, 2, 3, 2, 4, 5, 2, 2, 5, 5, 4, 5, 5, 0, 6, 5, 2,
       2, 5, 5, 2, 2, 5, 2, 0, 6, 3, 5, 5, 2, 2, 5,

Проверим нормальность количественных признаков, используя тест с лекции 

Если pvalue < 0.01, то будем считать, что распредление не нормально 

In [20]:
from scipy import stats
import statistics as st

In [106]:
def check_normal(name):
    print("pvalue for " + name)
    print("normality:", stats.normaltest(X[name], axis=0).pvalue) 
    print("lognormality:", stats.normaltest(np.log(X[name]), axis=0).pvalue) 

In [107]:
for col in cols:
    check_normal(col)

pvalue for rating
normality: 2.7653436520763265e-19
lognormality: 5.383238445438053e-14
pvalue for height
normality: 0.08864738596500196
lognormality: 0.04283437161023435
pvalue for weight
normality: 0.0044195881015143094
lognormality: 0.030643681067169058
pvalue for salary
normality: 2.9578135519485532e-25
lognormality: 5.093404263285901e-07


weight имеет нормальное распределение

In [110]:
from sklearn.model_selection import train_test_split

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, stratify=y, test_size=0.2)

In [108]:
from sklearn.ensemble import RandomForestClassifier

In [112]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

In [113]:
from collections import Counter

In [114]:
c = Counter(y)
c

Counter({2: 138, 4: 13, 5: 169, 3: 36, 0: 48, 6: 19, 1: 6})

Используем метрики, не чувствительные к балансу классов

In [115]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [122]:
recall_score(y_test,y_pred, average='micro')

0.7093023255813954

In [123]:
precision_score(y_test, y_pred, average='micro')

0.7093023255813954

In [125]:
f1_score(y_test, y_pred, average='micro')

0.7093023255813953